In [2]:
import os
import sys
import pandas as pd

root_path = os.path.abspath(os.path.join(os.getcwd(), '../..'))
sys.path.append(root_path)

In [5]:
from core.backtesting.optimizer import StrategyOptimizer
import json

optimizer = StrategyOptimizer(root_path=root_path)
dfs = []
for study_name in ["xgridt_2024-11-22"]:
    trials_df = optimizer.get_study_trials_df(study_name)
    trials_df["study_name"] = study_name
    dfs.append(trials_df)
trials = pd.concat(dfs)
def preprocess_trials_df(trials_df):
    df = trials_df.copy()
    df.columns = [col.replace("params_", "") if col.startswith("params_") else col for col in df.columns]
    df["executors"] = df["executors"].apply(lambda x: json.loads(x))
    df["config"] = df["config"].apply(lambda x: json.loads(x))
    df["trading_pair"] = df["config"].apply(lambda x: x['trading_pair'])
    df["from_timestamp"] = df["executors"].apply(
        lambda x: pd.to_datetime(pd.Series(x["timestamp"].values()).min(), unit="s"))
    df["to_timestamp"] = df["executors"].apply(
        lambda x: pd.to_datetime(pd.Series(x["close_timestamp"].values()).max(), unit="s"))
    return df
trials = preprocess_trials_df(trials)

trials['from_timestamp'] = pd.to_datetime(trials['from_timestamp'])
trials['to_timestamp'] = pd.to_datetime(trials['to_timestamp'])
trials['days_diff'] = (trials['to_timestamp'] - trials['from_timestamp']).dt.total_seconds() / 86400
trials['executors_per_day'] = trials['total_executors'] / trials['days_diff']
trials['volume_per_day'] = trials['total_volume'] / trials['days_diff']

In [6]:
len(trials)

324

In [7]:
trials.drop_duplicates(subset=["config"], inplace=True)
len(trials)

324

In [12]:
def get_best_trials(df):
    final_df = pd.DataFrame()

    for trading_pair in df['trading_pair'].unique():
        trading_pair_df = df[(df['trading_pair'] == trading_pair) & (df['executors'])]
        # filtered = trading_pair_df[
        #     (trading_pair_df['executors_per_day'] >= trading_pair_df['executors_per_day'].median()) &
        #     (trading_pair_df['volume_per_day'] >= trading_pair_df['volume_per_day'].median()) &
        #     (trading_pair_df['sharpe_ratio'] >= trading_pair_df['sharpe_ratio'].median()) &
        #     (trading_pair_df['max_drawdown_pct'] <= trading_pair_df['max_drawdown_pct'].median())
        # ]
        filtered = trading_pair_df.copy()

        if filtered.empty: 
            continue

        normalized_columns = ['executors_per_day', 'volume_per_day', 'sharpe_ratio', 'max_drawdown_pct']
        for col in normalized_columns:
            filtered[col] = filtered[col] / filtered[col].max()

        filtered['score'] = (
            filtered['executors_per_day'] *
            filtered['volume_per_day'] *
            filtered['sharpe_ratio'] *
            (1 / filtered['max_drawdown_pct'])
        )
        filtered['score'] = filtered['score'] / filtered['score'].max()
        filtered = filtered.dropna().sort_values(by='score', ascending=False).head(2)

        final_df = pd.concat([final_df, filtered[['study_name', 'config', 'trading_pair', 'max_drawdown_pct', 'executors_per_day', 'sharpe_ratio', 'volume_per_day', 'score']]])
    return final_df

# Call the function and pass the dataframe
best_trials = get_best_trials(trials)
best_trials

,study_name,config,trading_pair,max_drawdown_pct,executors_per_day,sharpe_ratio,volume_per_day,score
15,xgridt_2024-11-22,{'id': '3FP4CdaN54ZpNyQn3eeJVcU7zp9H991v3gQgMB...,PNUT-USDT,1.00184918,0.93066858,0.91698829,0.942927,1
19,xgridt_2024-11-22,{'id': '4ThfCTnaSkNP6kNMM5L2snrE8bqWSubk7RQLQo...,PNUT-USDT,1,0.97096389,0.84454117,0.97583731,0.99624638
278,xgridt_2024-11-22,{'id': '9P2ntvX8N6ZukDhiMyPSqcvicESxFeSPD2D3Z2...,1000BONK-USDT,1.38612492,0.80391615,0.84482159,0.86143862,1
292,xgridt_2024-11-22,{'id': 'CVdSxGyiv5vaJESj8vM3fMAu96NGhNQA8q6jWy...,1000BONK-USDT,1.60411394,0.80112168,1,0.81277762,0.96169484
8,xgridt_2024-11-22,{'id': '6RNBgz3LuAp77wGvGBRt8KW7vPTMFABoMQ1TuU...,1000PEPE-USDT,1.02743659,0.46885592,0.81598079,0.55643989,1
14,xgridt_2024-11-22,{'id': '5smjdVT4q1jbhNBsmssz8syhuFQF1BRN3hMFvQ...,1000PEPE-USDT,1,0.55963024,0.50850434,0.69240183,0.95098021
276,xgridt_2024-11-22,{'id': '2iTToUk9kGfHdfstfYEyAb4mG3yKUwTLv2gBKo...,HMSTR-USDT,1.09002635,0.53659322,0.76809898,0.65258653,1
78,xgridt_2024-11-22,{'id': 'GXv3UFKkM6Nk953knZdd6fDojWx2Ns8eNUw1QZ...,HMSTR-USDT,1.16169089,0.60944526,0.64470361,0.69187722,0.94835246
161,xgridt_2024-11-22,{'id': 'Eb1Ws9eVCELUkAAmEdXeRhQVJGS8SpBWokdD8B...,MOODENG-USDT,1.18491373,0.45099412,1,0.54185844,1
112,xgridt_2024-11-22,{'id': 'C4tA8xNTFb7GcUpn6NiCVr6tFTxi8KLcFZNd52...,MOODENG-USDT,1,0.43339231,0.72183617,0.53112257,0.80564665


In [31]:
import json
from core.services.backend_api_client import BackendAPIClient

configs = []
for index, row in best_trials.iterrows():
    config = row["config"]
    config["natr_length"] = 100
    config["leverage"] = 50
    config["controller_type"] = "generic"
    config["id"] = row["study_name"] + "_" + row["trading_pair"] + "_" + str(index)
    config["prominence_pct_peaks"] = 0.05
    config["distance_between_peaks"] = 100
    config["executor_activation_bounds"] = 0.001
    config["general_activation_bounds"] = 0.005
    config["max_ranges_by_signal"] = 1
    config["min_spread_between_orders"] = 0.0008
    config["min_order_amount"] = 2
    config["max_open_orders"] = 3
    config["max_orders_per_batch"] = 2
    config["order_frequency"] = 7
    config["close_position_on_signal_change"] = True
    config["grid_update_interval"] = None
    config["take_profit_mode"] = "original"
    config["take_profit_step_multiplier"] = 1
    config["global_stop_loss"] = 0.1
    # config.pop("cooldown_time")
    # config.pop("stop_loss")
    # config.pop("max_executors_per_side")
    # config.pop("take_profit")
    # config.pop("take_profit_order_type")
    # config.pop("trailing_stop")
    # config.pop("candles_config")
    configs.append(row["config"])

In [32]:
from core.utils import dump_dict_to_yaml

dump_dict_to_yaml(configs[0])

In [28]:
configs = best_trials["config"].tolist()
for config in configs:
    config["total_amount_quote"] = 100
    config["id"] = config["id"].replace("_", "-")

print(configs)
config_ids = [config["id"] + ".yml" for config in configs]
print(config_ids)

[{'id': 'xtreet-bb-binance-perpetual-1m-1000BONK-USDT-bb50-1.0-sl20.0-ts20.0-30.0', 'controller_name': 'xtreet_bb', 'controller_type': 'directional_trading', 'total_amount_quote': 100, 'manual_kill_switch': None, 'candles_config': [{'connector': 'binance_perpetual', 'trading_pair': '1000BONK-USDT', 'interval': '1m', 'max_records': 70}], 'connector_name': 'binance_perpetual', 'trading_pair': '1000BONK-USDT', 'max_executors_per_side': 1, 'cooldown_time': 0, 'leverage': 20, 'position_mode': 'HEDGE', 'stop_loss': 0.2, 'take_profit': 0.02, 'time_limit': None, 'take_profit_order_type': 2, 'trailing_stop': {'activation_price': 0.2, 'trailing_delta': 0.3}, 'candles_connector': 'binance_perpetual', 'candles_trading_pair': '1000BONK-USDT', 'interval': '1m', 'bb_length': 50, 'bb_std': 1.0, 'bb_long_threshold': 0.0, 'bb_short_threshold': 1.0, 'dca_spreads': [-1e-08, 0.6000000000000001], 'dca_amounts_pct': [1, 1.0], 'dynamic_order_spread': True, 'dynamic_target': True, 'min_stop_loss': 0.003, 'max_

In [29]:
client = BackendAPIClient(
    host="localhost",
    port=8000,
)
for config in configs:
    await client.add_controller_config(config)


In [30]:
await client.deploy_script_with_controllers(
        bot_name="test_1",
        controller_configs=config_ids,
        image_name="dardonacci/hummingbot:latest",
        max_global_drawdown=50,
        max_controller_drawdown=20,
    )